# Simulate Data for Contact Tracing. 

In [5]:
# Useful imports
from uuid import uuid4 as v4
import random
import json
import boto3
from datetime import datetime, tzinfo, timedelta

In [6]:
# weighted random selection
## matches are the number of interactions every hour. (its a weighted selection between 0 and 50, where less is more probable.)
matches = lambda : random.choice([0]+[random.randrange(i) for i in range(1,random.choice([1,1,1,1,1,1,10,10,25,50]))])

# here we simulate uuid32 for 15 days, 24 hours, where data is stored every hour. 
days = lambda : [[[v4().hex for i in range(matches())] for i in range(24)] for i in range(15)]

In [7]:
# simple function to generate current UTC time in ISO format.
class simple_utc(tzinfo):
    def tzname(self,**kwargs):
        return "UTC"
    def utcoffset(self, dt):
        return timedelta(0)
gettime = lambda : datetime.utcnow().replace(tzinfo=simple_utc()).isoformat()[:-9]+'Z'

In [8]:
# This is what our database Item looks like.
createDbItem = lambda: {
    'PutRequest': {
        'Item': {
            "Partition": "Contact",
            "CreationTime": gettime(),
            "Data": {
                "days": days(),
                "startDay": 18521
            }
        }
    }
}

In [9]:
## HEAVY OPERATION
# this where 5000 records are generated.

li = [[createDbItem() for i in range(25)] for i in range(2000)]

In [11]:
with open('./data.json', 'w') as f:
    f.write(json.dumps(li))